## model.py

##### Accuracy = approx 99%

***

In [3]:
import tensorflow
import pandas
import csv

In [4]:
#single forward slash for the directory's name because I do not use windows.

import requests
from bs4 import BeautifulSoup
url='https://karki23.github.io/Weather-Data/assignment.html'
a=requests.get(url)
r=BeautifulSoup(a.content, "lxml")
cities=r.find_all('a')
import csv
file_name="dataset/"+"All_Cities."+"csv"
f=open(file_name, "a", newline="")
writer=csv.writer(f)
writer.writerow(['Date','Location','MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','WindGustDir','WindGustSpeed','WindDir9am','WindDir3pm','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Cloud9am	','Cloud3pm','Temp9am','Temp3pm','RainToday','RISK_MM','RainTomorrow'])
for x in range(len(cities)):
    s=cities[x].get('href')[0:len(cities[x])-5:]    
    new_url='https://karki23.github.io/Weather-Data/'+cities[x].get('href')
    new_a=requests.get(new_url)
    new_r=BeautifulSoup(new_a.content, "lxml")
    row=new_r.find_all('tr')
    row.pop(0) 
    writer=csv.writer(f)
    for i in row:    
        column=i.find_all('td')
        new_column=[j.text for j in column]
        writer.writerow(new_column)
f.close()

In [5]:
import os
cwd = os.getcwd()
print(cwd)

/Users/sakshishetty/Desktop/Python


In [6]:
os.chdir("/Users/sakshishetty/Desktop/Python/dataset")  #CHANGE ACCORDING TO YOUR SYSTEM
cwd = os.getcwd()
print(cwd)

/Users/sakshishetty/Desktop/Python/dataset


In [7]:
dataset = pandas.read_csv('All_Cities.csv')      #File consisting of data of all 49 cities
dataset = dataset.drop(columns =['Date','Location'])

(read the comments)

In [8]:
'''from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataset['RainToday']=encoder.fit_transform(dataset['RainToday'])
I initially tried using this method for all the string data, but I got an error I wasn't able to fix, 
so I opted for the method below.
'''

dataset['WindGustDir'] = pandas.Categorical(dataset['WindGustDir'])
dataset['WindGustDir'] = dataset.WindGustDir.cat.codes
dataset['WindDir9am'] = pandas.Categorical(dataset['WindDir9am'])
dataset['WindDir9am'] = dataset.WindDir9am.cat.codes
dataset['WindDir3pm'] = pandas.Categorical(dataset['WindDir3pm'])
dataset['WindDir3pm'] = dataset.WindDir3pm.cat.codes
dataset['RainTomorrow'] = pandas.Categorical(dataset['RainTomorrow'])
dataset['RainTomorrow'] = dataset.RainTomorrow.cat.codes
dataset['RainToday'] = pandas.Categorical(dataset['RainToday'])
dataset['RainToday'] = dataset.RainToday.cat.codes

dataset[['WindGustDir','WindDir9am','WindDir3pm']] = dataset[['WindGustDir','WindDir9am','WindDir3pm']].astype(float)

#to get rid of negative values
import numpy as np
dataset[['WindDir9am']] = np.where(dataset[['WindDir9am']]<0, 0, dataset[['WindDir9am']])
dataset[['WindGustDir']] = np.where(dataset[['WindGustDir']]<0, 0, dataset[['WindGustDir']])
dataset[['WindDir3pm']] = np.where(dataset[['WindDir3pm']]<0, 0, dataset[['WindDir3pm']])
dataset[['RainToday']] = np.where(dataset[['RainToday']]<0, 0, dataset[['RainToday']])

#filling the missing values with mean
dataset.fillna(dataset.mean(), inplace=True)

In [9]:
seed = 42
np.random.seed(seed)

In [10]:
xtrain = dataset.drop(columns=['RainTomorrow'])
ytrain = dataset['RainTomorrow']

In [11]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
n = xtrain.shape[1]  #number of columns in training data

model.add(Dense(250, activation='sigmoid', input_shape=(n,)))
model.add(Dense(250, activation='sigmoid'))
model.add(Dense(250, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
#accuracy seems to vary vastly based on the usage of sigmoid or relu or softmax

Using TensorFlow backend.
W0723 23:08:55.587437 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 23:08:55.628325 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 23:08:55.644784 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [12]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

W0723 23:08:59.556368 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 23:08:59.578849 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.



In [13]:
model.fit(xtrain, ytrain, validation_split=0.2, epochs=10)

W0723 23:09:01.398101 4622742976 deprecation.py:323] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0723 23:09:01.424935 4622742976 deprecation_wrapper.py:119] From /Users/sakshishetty/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 113754 samples, validate on 28439 samples
Epoch 1/10
113754/113754 [==============================] - 5s 41us/step - loss: 0.1360 - acc: 0.9494 - val_loss: 0.0153 - val_acc: 0.9966
Epoch 2/10
113754/113754 [==============================] - 4s 37us/step - loss: 0.0426 - acc: 0.9849 - val_loss: 0.0102 - val_acc: 0.9984
Epoch 3/10
113754/113754 [==============================] - 4s 39us/step - loss: 0.0360 - acc: 0.9872 - val_loss: 0.0800 - val_acc: 0.9786
Epoch 4/10
113754/113754 [==============================] - 4s 39us/step - loss: 0.0352 - acc: 0.9873 - val_loss: 0.0093 - val_acc: 0.9961
Epoch 5/10
113754/113754 [==============================] - 5s 41us/step - loss: 0.0353 - acc: 0.9884 - val_loss: 0.0248 - val_acc: 0.9898
Epoch 6/10
113754/113754 [==============================] - 5s 40us/step - loss: 0.0337 - acc: 0.9892 - val_loss: 0.0440 - val_acc: 0.9826
Epoch 7/10
113754/113754 [==============================] - 5s 40us/step - loss: 0.0320 - acc: 0.9902 - val_loss: 0

***